In [1]:
from pyspark.sql import *
from pyspark.sql.types import *

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1617673582929_0007,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark_df=spark.read.csv('wasb:///input/worldometer_data.csv', header=True, inferSchema=True)

In [3]:
spark_df.describe()
spark_df.show()

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious/Critical|TotCases/1Mpop|Deaths/1Mpop|TotalTests|Tests/1Mpop|           WHORegion|
+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+--------------------+
|           USA|North America| 331198130|   5032179|    null|     162804|     null|       2576668|        null|    2292707|           18296|         15194|       492.0|  63139605|     190640|            Americas|
|        Brazil|South America| 212710692|   2917562|    null|      98644|     null|       2047660|        null|     771258|            8318|        

In [6]:
from pyspark.sql.functions import isnan, when, col, count, isnull 
spark_df.select([count(when(isnan(c), c)).alias(c) for c in spark_df.columns]).show()
spark_df.select([count(when(col(c).isNull(), c)).alias(c) for c in spark_df.columns]).show()

+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+---------+
|Country/Region|Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious/Critical|TotCases/1Mpop|Deaths/1Mpop|TotalTests|Tests/1Mpop|WHORegion|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+---------+
|             0|        0|         0|         0|       0|          0|        0|             0|           0|          0|               0|             0|           0|         0|          0|        0|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+---------+

+--------

In [8]:
spark_df_missing = spark_df.na.fill(0)

In [9]:
spark_df_missing.select([count(when(isnan(c), c)).alias(c) for c in spark_df_missing.columns]).show()
spark_df_missing.select([count(when(col(c).isNull(), c)).alias(c) for c in spark_df_missing.columns]).show()

+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+---------+
|Country/Region|Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious/Critical|TotCases/1Mpop|Deaths/1Mpop|TotalTests|Tests/1Mpop|WHORegion|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+---------+
|             0|        0|         0|         0|       0|          0|        0|             0|           0|          0|               0|             0|           0|         0|          0|        0|
+--------------+---------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+--------------+------------+----------+-----------+---------+

+--------

In [12]:
spark_df_missing = spark_df_missing.drop('NewCases', 'NewDeaths','NewRecovered','WHORegion','TotCases/1Mpop','Deaths/1Mpop','Tests/1Mpop','WHORegion')

In [15]:
spark_df_missing.show()

+--------------+-------------+----------+----------+-----------+--------------+-----------+----------------+----------+
|Country/Region|    Continent|Population|TotalCases|TotalDeaths|TotalRecovered|ActiveCases|Serious/Critical|TotalTests|
+--------------+-------------+----------+----------+-----------+--------------+-----------+----------------+----------+
|           USA|North America| 331198130|   5032179|     162804|       2576668|    2292707|           18296|  63139605|
|        Brazil|South America| 212710692|   2917562|      98644|       2047660|     771258|            8318|  13206188|
|         India|         Asia|1381344997|   2025409|      41638|       1377384|     606387|            8944|  22149351|
|        Russia|       Europe| 145940924|    871894|      14606|        676357|     180931|            2300|  29716907|
|  South Africa|       Africa|  59381566|    538184|       9604|        387316|     141264|             539|   3149807|
|        Mexico|North America| 129066160

In [19]:
spark_df_missing = spark_df_missing.withColumnRenamed('Country/Region', 'Country')
spark_df_missing = spark_df_missing.withColumnRenamed('Serious/Critical', 'Critical')

In [20]:
spark_df_missing.show()

+------------+-------------+----------+----------+-----------+--------------+-----------+--------+----------+
|     Country|    Continent|Population|TotalCases|TotalDeaths|TotalRecovered|ActiveCases|Critical|TotalTests|
+------------+-------------+----------+----------+-----------+--------------+-----------+--------+----------+
|         USA|North America| 331198130|   5032179|     162804|       2576668|    2292707|   18296|  63139605|
|      Brazil|South America| 212710692|   2917562|      98644|       2047660|     771258|    8318|  13206188|
|       India|         Asia|1381344997|   2025409|      41638|       1377384|     606387|    8944|  22149351|
|      Russia|       Europe| 145940924|    871894|      14606|        676357|     180931|    2300|  29716907|
|South Africa|       Africa|  59381566|    538184|       9604|        387316|     141264|     539|   3149807|
|      Mexico|North America| 129066160|    462690|      50517|        308848|     103325|    3987|   1056915|
|        P

In [26]:
spark_df_missing.write.csv('wasb:///input/input_hive_data/input/')

In [ ]:
spark_df_missing.write.saveAsTable("hvac")